In [1]:
from Pianki.Analiza_pianek.Analiza_rodziny import Analiza_Rodziny
from Pianki.Modele_pianek import *
from Pianki.Raporty_do_zamowien.Komponowanie_zamowienia import Raport_zamowionych_pianek_i_owat
from Pianki.Raporty_do_zamowien import *
import warnings
warnings.filterwarnings('ignore')

1. Bryly do zamówienia:
* `Analiza_Rodziny(SAMOA_P).Bryly_do_zamowienia(wszystkie_bryly=True, lista_korekty_zam=True)`
*   `SAMOA_P` klasa opisujaca model do zamówienia
* `Bryly_do_zamowienia(wszystkie_bryly=True, lista_korekty_zam=True)` Funkcja wywołująca listę brył do zamówienia
2. Przypisanie brył do zmiennych i obliczenie obiętosci zamówienia
* `zsam, sam =Analiza_Rodziny(SAMOA_P).Bryly_do_zamowienia(wszystkie_bryly=True, korekta_zam={'1,5': 70.0, 'Pufa 100x82': 20.0,})`
* `zsam` tabelka z zamówionymi bryłmi
* `sam` klasa z piankami i obiętościami
* `korekta_zam={'1,5': 70.0, 'Pufa 100x82': 20.0,}` korekta listy zamówienia, 

In [2]:
Analiza_Rodziny(AVANT_P).Bryly_do_zamowienia(korekta_zam={'[1]': 4.0, '[3,5]': 12.0, '1,5': 8.0, '2': 2.0, '2,5': 7.0, '3': 1.0, '2][': 14.0, '2,5][': 1.0, 'N': 28.0, 'L][': 3.0, 'LM][': 14.0, 'Pufa 112x84': 10.0, 'Pufa 112x97': 15.0, 'Pufa 64x64': 2.0})

(                KOD           OPIS  DO_ZAMOWIENIA
 0   16.009.10003.60      AVANT [1]            4.0
 1   16.009.35003.60    AVANT [3,5]           12.0
 2   16.009.15000.60      AVANT 1,5            8.0
 3   16.009.20000.60        AVANT 2            2.0
 4   16.009.25000.60      AVANT 2,5            7.0
 5   16.009.30000.60        AVANT 3            1.0
 6   16.009.20004.60      AVANT 2][           14.0
 7   16.009.25004.60    AVANT 2,5][            1.0
 8   16.009.66000.60        AVANT N           28.0
 9   16.009.63004.60      AVANT L][            3.0
 10  16.009.64004.60     AVANT LM][           14.0
 11  16.009.71100.60  AVANT P112x84           10.0
 12  16.009.71000.60  AVANT P112x97           15.0
 13  16.009.70500.60   AVANT P64x64            2.0,
 AVANT Vol Pianpol: 30.79m3, AVANT Vol OLTA: 7.03m3)

In [3]:
zsto, sto = Analiza_Rodziny(STONE_P).Bryly_do_zamowienia(wszystkie_bryly=True, korekta_zam={
    '1,5': 40,
    # '2,5': 6.0,
    '3': 30.0,
    # '4': 15.0,
    # '5': 0.0,
    # 'ON': 15.0,
    'NW140x150': 25.0,
    'NW MEM': 40.0,
    'WN MEM': 60.0,
    '[LH': 10.0,
    # 'E30': 2.0,
    # 'P106x106': 1.0,
    # 'P106x82': 4.0,
    # 'P82x82': 3.0,
    # 'Z50': 8.0,
    # 'Z70': 0.0,
    'STONE ][': 50.0,
    # 'CUPIDO ][': 0.0
 })

zeli, eli = Analiza_Rodziny(ELIXIR_P).Bryly_do_zamowienia(wszystkie_bryly=True, korekta_zam={
    # '2,5': 5.0,
    '3': 15.0,
    '3,5': 30.0,
    '4': 40.0,
    'N': 40.0,
    '[': 65.0,
    'Pufa 98x120': 10.0,
    # 'Pufa 98x100': 0.0
 })

In [4]:
sto.pianpol_VOL, sto.ciech_VOL, sto.vita_VOL

(45.76474999999999, 0, 0)

In [5]:
zmod = [zsto, zeli]

In [6]:
df, _ = Raport_zamowionych_pianek_i_owat(zmod)#, "PIANPOL_2421.xlsx")
df.columns

O1 zielona: 5 rolek
O2 niebieska: 0 rolek
O3 czerwona: 22 rolek


Index(['KOD', 'OPIS', 'DO_ZAMOWIENIA', 'ZIELONA', 'NIEBIESKA', 'CZERWONA',
       'ŻÓŁTA', 'W3'],
      dtype='object')

In [8]:

# Dodaj_pozycje_do_ZAM_PIANKI(2420, "P", "3_24", zsam, sam, "24/0516", nr_partii="20/01")#, DODAJ_DO_BAZY=True)
# Dodaj_pozycje_do_ZAM_PIANKI(2420, "P", "2_24", zhud, hud, "24/0516", nr_partii="20/01")#, DODAJ_DO_BAZY=True)
# Dodaj_pozycje_do_ZAM_PIANKI(2420, "P", "3_24", zcoc, coc, "24/0516", nr_partii="20/01")#, DODAJ_DO_BAZY=True)
# Dodaj_pozycje_do_ZAM_PIANKI(2420, "P", "3_24", zcal, cal, "24/0516", nr_partii="20/01")#, DODAJ_DO_BAZY=True)





In [9]:
# gre = pobierz_zamowienie_z_ZAM_PIANKI(2411, GREY_C)
# raport_dostarczonych_pianek(gre, "24/0327",)

In [10]:
# Wozki_do_dostawy("PIANPOL", "24/0516", drukuj_do_excel=True)

In [11]:
# raport_dostarczonych_pianek(sam, "24/0516",True)
# raport_dostarczonych_pianek(hud, "24/0516",True)
# raport_dostarczonych_pianek(coc, "24/0516",True)
# raport_dostarczonych_pianek(cal, "24/0516",True)




In [12]:
# zlecenia_produkcyjne_pianki_owaty("SAMOA", "3_24", "20/01")
# zlecenia_produkcyjne_pianki_owaty("HUDSON", "2_24", "20/01")
# zlecenia_produkcyjne_pianki_owaty("COCO", "3_24", "20/01")
# zlecenia_produkcyjne_pianki_owaty("CALYPSO", "3_24", "20/01")
# zlecenia_produkcyjne_pianki_owaty("WILLOW", "2_24", "20/01")


# zlecenia_produkcyjne_pianki_kompletacja("SAMOA", "3_24", "20/01")
# zlecenia_produkcyjne_pianki_kompletacja("HUDSON", "2_24", "20/01")
# zlecenia_produkcyjne_pianki_kompletacja("CALYPSO", "3_24", "20/01")
# zlecenia_produkcyjne_pianki_kompletacja("COCO", "3_24", "20/01")


In [13]:
# ava.olta.to_excel("AVANT_MEMORY.xlsx")

In [14]:
from Modele_db import engine, text
import openpyxl

def naklejki_excell():
    with engine.begin() as conn:
        zp = conn.execute(text(f"""SELECT MODEL, NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE,
                        NR_PARTII from ZAM_PIANKI WHERE ZAM1 in('24/0486') --AND OPIS NOT LIKE '%AVANT%'"""))
    
    zam_pianki = zp.fetchall()
    zam_pianki

    wb = openpyxl.Workbook()
    sheet = wb.active

    #NALKEJKI PIANEK
    for row in zam_pianki:
       
        for i in range(row[3]):
           
            sheet.append([f"NR PARTII: {row[4]}", f"NR KOMPLETACJI: {row[1]}", f"NR PACZKI: {i+1}/{row[3]}"])
            sheet.append([row[0]]) #MODEL
            sheet.append([row[2].replace(row[0], "").strip()]) #BRYLA
            sheet.append(["PACZKA JAKOŚĆ NR PRAC", "UWAGI", "NR PAKUJĄCEGO"])

   
    #NAKLEJKI OWAT
    for row in zam_pianki:
       
        for i in range(row[3]):
           
            sheet.append([f"NR PARTII: {row[4]}", f"OWATY", f""])
            sheet.append([row[0]]) #MODEL
            sheet.append([row[2].replace(row[0], "").strip()]) #BRYLA
            sheet.append(["UWAGI", "NR PAKUJĄCEGO"])

    wb.save("nakl.xlsx")

naklejki_excell()
